In [1]:
%load_ext sql
import socket
import json
import os

In [1]:
%%html
<style>
td {text-align: left !important; valign: left !important;}
</style>

# Database migration with the Liberatii Data Platform

The Liberatii Data Platform virtualizes a PostgreSQL database to enable Oracle applications to be migrated to PostgreSQL *without modification*. This notebook contains a tutorial and demonstration of the deployment and use of the Liberatii Data Platform to migrate and test an example Oracle database.

## Architecture

The Liberatii Data Platform uses the Liberatii Gateway to virtualize Azure Databases. This allows
applications built for Oracle to use these databases with **no code changes** and
**no new tests** required.

<img src="Architecture.png" style="margin:auto" alt="Diagram of Liberatii Platform Architecture" title="Architecture"/>

### Change-Data-Capture and Parallel Test Execution

Production workloads can be tested against up-to-date Azure replicas of the Oracle Database
virtualised through Liberatii Gateway using the Change-Data-Capture (CDC) and Parallel Test Execution
features of the Liberatii Data Platform.

<img src="Replay.png" style="margin:auto" alt="Diagram of Workload Replay" title="Replay"/>

This technique provides full confidence for a successful migration **without additional tests**
before any applications are re-targeted to the Liberatii Gateway.

## Demonstration

This notebook uses the
[Oracle H.R. schema](https://docs.oracle.com/cd/E11882_01/server.112/e40540/tablecls.htm#CNCPT111)
to demonstrate the migration of the data and execution of existing Oracle PL/SQL code via
the Liberatii Gateway on a PostgreSQL database.

The H.R. schema provides a basis for queries that use several Oracle-specific features that
are virtualized through the Liberatii Gateway:

<img src="Schema.gif" style="margin:auto" alt="Diagram of the HR Schema" title="Schema"/>

## Migrating an application

This demonstration will migrate the schema and correspdoning SQL with Liberatii Data Platform though the following steps:

1. **Deployment**<br/>
   The deployment of an Azure Managed Application to provide the Liberatii Data Platform
   
3. **Migration**<br/>
   The migration of the schema and data from the Oracle database to PostgreSQL via the Liberatii Gateway
   
5. **Synchronisation**<br/>
   Setup of a Change Data Capture pipeline to synchronize the Oracle and PostgreSQL databases
   
7. **Replay testing**<br/>
   Testing of an Oracle Workload Replay for **performance** and **correctness** against both the Oracle and PostgreSQL databases to verify proper
   operation
   
9. **Switch over**<br/>
   Replacing the database driver and retargeting the application to use the Liberatii Gateway

# Set up

The following cell contains the connection information for the Oracle (Source) and PostgreSQL (Target) databases. It is initially set up to migrate the Oracle demo H.R. schema and expects the given users to exist with the required permissions on the specified databases.

In [3]:
## Hostnames of the databases
PG_HOST='postgres'
ORACLE_HOST='oracle'

## Connection data
DB='pdborcl'
USER='HR'
PSWD='hr'
ORACLE_PORT=1521
PG_PORT=5432
LGW_PORT=15432

## Connection strings (derived from the above connection data)
ORACLE_CONN_STR=f'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={ORACLE_HOST})(PORT={ORACLE_PORT}))(CONNECT_DATA=(SERVICE_NAME={DB})))'
PG=f'postgresql://{USER}:{PSWD}@{PG_HOST}:{PG_PORT}/{DB}'
ORACLE=f'oracle://{USER}:{PSWD}@{ORACLE_CONN_STR}'

print(f"""
Connection settings:

    PostgreSQL:             
      {PG}
    Oracle:                 
      {ORACLE}
""")


Connection settings:

    PostgreSQL:             
      postgresql://HR:hr@postgres:5432/pdborcl
    Oracle:                 
      oracle://HR:hr@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=oracle)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=pdborcl)))



This Notebook uses the SQL extension so connections the databases can be queried directly. The following cells check the connections to Oracle and PostgreSQL are working:

In [4]:
%%sql {ORACLE}
-- Find version information for the Oracle database
select banner FROM v$version

0 rows affected.


banner
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production


In [5]:
%%sql {PG}
-- Find version information for the PostgreSQL database
select version()

1 rows affected.


version
"PostgreSQL 14.9 (Debian 14.9-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit"


# Stage 1: Deploy Liberatii Data Platform and Gateway

The Liberatii Data Platform and Gateway are available as a single Azure Managed Application. In this notebook we will use the Azure command line to deploy the managed application.

## Login

The following cell ensures that the user is logged into Azure using the `az` command line tool.

In [6]:
account=!az account list
result = None
try:
    result = json.loads("".join(account))
except json.JSONDecodeError:
    if os.environ.get('NOAZURE', 0) == 0:
        !az login -o none
    account=!az account list
    try:
        result = json.loads("".join(account))
    except json.JSONDecodeError:
        print("Not logged in: " + "\n".join(account))

if result is not None:
    print("Logged in, available subscriptions:")
    print("\n".join([
        f"  {e['id']}: {e['name']}"
        for e in json.loads("".join(account))
        if e['state'] == 'Enabled'
    ]))

Logged in, available subscriptions:
  1b29918b-b8fa-41b6-9c70-c58b9c06c4a5: Microsoft Partner Network


## Parameters

The next cell defines the parameters for creation of the Liberatii managed application. We will define a new resource group for this.

In [7]:
RESOURCE_GROUP='NotebookTest'

definitionid = !az managedapp definition show \
    --subscription "Pay-As-You-Go Dev/Test" \
    -g vmonlypgtransmanapp0v123 -n vmonlypgTranslator_0v123 \
    --query id --output tsv
subscriptionid = !az account show --query id --output tsv
resourceid = f"/subscriptions/{subscriptionid[0]}/resourceGroups/{RESOURCE_GROUP}-mrg"
params = json.dumps([{
    "postgresqlHost":       {"value": PG_HOST},
    "postgresqlPassword":   {"value": USER},
    "postgresqlUsername":   {"value": PSWD},
    "interconnectPassword": {"value": "someSecureString"},
    "infoQueryPassword":    {"value": "someSecureString"},
    "customerParameters":   {"value": ""},
    "poolMode":             {"value": "session"},
    "port":                 {"value": LGW_PORT},
    "virtualMachinePrefix": {"value": "mftde"},
    "virtualNetworkRange":  {"value": "192.168.1.0/24"},
    "virtualMachineSize":   {"value": "Standard_F2s_v2"},
}])

print('Parameters:\n{\n%s\n}' % 
      ',\n'.join([
          f'''  {f"'{k}':":<24} {v}'''
          for k, v in json.loads(params)[0].items()
      ]))

Parameters:
{
  'postgresqlHost':        {'value': 'postgres'},
  'postgresqlPassword':    {'value': 'HR'},
  'postgresqlUsername':    {'value': 'hr'},
  'interconnectPassword':  {'value': 'someSecureString'},
  'infoQueryPassword':     {'value': 'someSecureString'},
  'customerParameters':    {'value': ''},
  'poolMode':              {'value': 'session'},
  'port':                  {'value': 15432},
  'virtualMachinePrefix':  {'value': 'mftde'},
  'virtualNetworkRange':   {'value': '192.168.1.0/24'},
  'virtualMachineSize':    {'value': 'Standard_F2s_v2'}
}


## Creation

The resource group and managed application can now be created.

In [ ]:
groups=!az group list --query "[?name=='{RESOURCE_GROUP}'].id"
try:
    matching = len(json.loads("".join(groups)))
except json.JSONDecodeError:
    matching = -1
if os.environ.get('NOAZURE', 0) != 0:
    print("Skipping azure deployment")
if matching > 0:
    print("Group already exists, skipping deployment")
elif matching == -1:
    print("Azure error: " + "".join(groups))
else:
    !az group create -g {RESOURCE_GROUP} -l ukwest
    !az managedapp create -n TestTranslator -g {RESOURCE_GROUP} \
        --location ukwest \
        --managedapp-definition-id "{definitionid[0]}" \
        --kind ServiceCatalog \
        --managed-rg-id "{resourceid}" \
        --parameters '{params}'

## Connections

The following cell will construct connection strings for the newly deployed Liberatii Data Platform and Gateway.

If the application was not deployed other values may be used to access the Liberatii Gateway and Data Platform using the `LGW_HOST` and `PLATFORM` variables directly.

### A note on drivers

In this example we are using the SQLAlchemy `postgres://` driver to connect to the Liberatii Gateway. The Liberatii Gateway also supports the use of OCI, ODBC, JDBC, Pro\*COBOL and Pro\*C through a drop-in replacement driver that can be used to provide connectivity to 3rd-party applications.

In [8]:
## Use these values if the deployment stage was skipped
LGW_HOST='pgtranslator'
PLATFORM='migration'

## ...otherwise, if there is a deployment, collect the address
ips=!az network public-ip list -g {RESOURCE_GROUP}-mrg --query [].ipAddress
if ips[0][0] == '[':
    LGW_HOST=json.loads("".join(ips))[0]
    PLATFORM=LGW_HOST
else:
    print("Ignoring resource group: \n  " + '\n  '.join(ips))
    
LGW=f'postgresql://{USER}:{PSWD}@{LGW_HOST}:{LGW_PORT}/{DB}'
API_PREFIX=f"http://{socket.gethostbyname(PLATFORM)}:3000"

print(f"""
Connection settings:

    Liberatii Gateway:
      {LGW}
    Liberatii Data Platform:
      {API_PREFIX}/api
    
The API may be opened in a browser for reference.
""")

Ignoring resource group: 
  ERROR: (ResourceGroupNotFound) Resource group 'NotebookTest-mrg' could not be found.
  Code: ResourceGroupNotFound
  Message: Resource group 'NotebookTest-mrg' could not be found.

Connection settings:

    Liberatii Gateway:
      postgresql://HR:hr@pgtranslator:15432/pdborcl
    Liberatii Data Platform:
      http://192.168.32.7:3000/api
    
The API may be opened in a browser for reference.



## Test

With the Liberatii Gateway deployed (or otherwise available) we can now test the connection.

The following command uses Oracle SQL with will be translated by the virtualisation layer to access the PostgreSQL database.

In [9]:
%%sql {LGW}
select banner FROM v$version

1 rows affected.


BANNER
"PostgreSQL 14.9 (Debian 14.9-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit"


# Stage 2. Migration

The following section shows the migration of the schema and data from the Oracle database to PostgreSQL. This uses the Liberatii Data Platform to perform the following steps:

1. Configuration, connection setup and initialisation
3. Schema Migration
4. Data Migration
5. Testing and verification

All of these steps **can be performed in parallel** across multiple threads and connections to cope with large volumes of data.

## Configuration

The following cells will construct the required connection information. Each connection (Oracle, PostgreSQL and the Liberatii Gateway) will be created in turn.

### Oracle

In [10]:
!curl -s {API_PREFIX}/connection -H 'Content-Type: application/json' \
   -d '{{"type":"Oracle","connectionString":"{ORACLE_CONN_STR}","user":"{USER}","password":"{PSWD}","id":1}}'

{"type":"Oracle","connectionString":"(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=oracle)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=pdborcl)))","user":"HR","password":"hr","id":1}

### PostgreSQL

In [11]:
!curl -s {API_PREFIX}/connection -H 'Content-Type: application/json' \
  -d '{{"type":"PostgreSQL","host":"{PG_HOST}","port":5432,"database":"{DB}","user":"{USER}","password":"{PSWD}","id":2}}'

{"type":"PostgreSQL","host":"postgres","port":5432,"database":"pdborcl","user":"HR","password":"hr","id":2}

### Liberatii Gateway:

In [12]:
!curl -s {API_PREFIX}/connection -H 'Content-Type: application/json' \
  -d '{{"type":"LGW","host":"{LGW_HOST}","port":{LGW_PORT},"database":"{DB}","user":"{USER}","password":"{PSWD}", "id":3}}'

{"type":"LGW","host":"pgtranslator","port":15432,"database":"pdborcl","user":"HR","password":"hr","id":3}

### Configuration Parameters

The following configuration parameters are used for this demonstration. Please check the reference document to get all the available parameters.

* `dataOnePass`<br/>
  Don't use staging files or tables, just transfer all the data from the source to the target
* `user`<br/>
  List of schemas to transfer between the sources and target target
* `verbose`<br/>
  Log verbosity
* `eraseOnInit`<br/>
  Delete everything from the schema in the init stage, this is useful if we often restart the migration.

In [13]:
result = !curl -s {API_PREFIX}/config -H 'Content-Type: application/json' \
  -d '{{"dataOnePass": true, "users":["{USER}"], "verbose":4, "eraseOnInit":true}}'
print(json.dumps(json.loads(result[0]), indent=True))

{
 "message": "Config has been set successfully",
 "config": {
  "dataOnePass": true,
  "verbose": 4,
  "useCopy": true,
  "dataIterations": -1,
  "useWrapper": true,
  "useNative": false,
  "useUnlogged": false,
  "stat": true,
  "statDB": false,
  "rowsBuf": 1000,
  "lightCheck": false,
  "dataChunkSize": -1,
  "bigTablesFirst": true,
  "debReverseOrder": false,
  "cli": true,
  "rmStagingFiles": true,
  "parTables": true,
  "hashType": "murmur",
  "simulateUnsupportedTypes": true,
  "blobStreams": true,
  "clobStreams": false,
  "blobImmed": false,
  "clobImmed": false,
  "dumpCopy": false,
  "dryRun": false,
  "noBlobs": false,
  "idCol": "rowid",
  "idColByTable": {},
  "activeSqlFiles": [],
  "numTries": 10,
  "maxWait": 60000,
  "ignoreTrim": true,
  "removeLastFetchFirst": false,
  "checkMetaData": true,
  "users": [
   "HR"
  ],
  "linkedServers": [],
  "stages": {},
  "workloadsFiles": "",
  "eraseOnInit": true,
  "debezium": "http://kafka-connect:8083/",
  "replayDir": "/tmp

### Initialisation

This is an initialisation operation. The framework runs simple assessments and stores the results in the target database. The operation is asynchronous: it just schedules the operation and exists immediately.

In [14]:
result = !curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "init" }}'
print(json.dumps(json.loads(result[0]), indent=True))

{
 "message": "Config has been set successfully.",
 "config": {
  "title": "init",
  "status": "Running",
  "messages": [],
  "progress": 0
 }
}


Using this operation we can get the current state of the currently running operation:

In [15]:
result = !curl -s {API_PREFIX}/operation?pager=1 -H 'Content-Type: application/json'
print(json.dumps(json.loads(result[0]), indent=True))

{
 "title": "init",
 "status": "Running",
 "messages": [],
 "progress": 0
}


And the next operation is the same as the previous one, except it waits until the current operation ends.

In [16]:
result = !curl -s -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'
print(json.dumps(json.loads(result[0]), indent=True))

{
 "title": "init",
 "status": "Running",
 "messages": [
  {
   "level": "Info",
   "message": "\n^g^+DONE! \ud83d\udc4d^:\n"
  },
  {
   "level": "Info",
   "message": [
    [
     "Type",
     "I",
     "Total"
    ],
    [
     "VIEW",
     "1",
     "1"
    ],
    [
     "TABLE",
     "7",
     "7"
    ],
    [
     "PROCEDURE",
     "2",
     "2"
    ],
    [
     "SEQUENCE",
     "3",
     "3"
    ],
    [
     "INDEX",
     "11",
     "11"
    ],
    [
     "^+Total:^:",
     "24",
     "^+24^"
    ]
   ]
  }
 ],
 "progress": 0
}


## Schema migration

After `init` stage is complete we have all the necessary information for migrating the database schema.

The migration metadata is stored in the PostgreSQL database allowing us to run various queries on it. For example, we can query the number of objects of each type by running with the query:


In [17]:
%%sql {PG}
select count(*), type, stage, error from dbt.migration_objects group by type, stage, error

5 rows affected.


count,type,stage,error
11,INDEX,I,None
2,PROCEDURE,I,None
7,TABLE,I,None
3,SEQUENCE,I,None
1,VIEW,I,None


If there were any errors during the execution of any stage, they will be displayed in `error` stage. The errors can be manually fixed by modifying `ddl1`, `ddl2` columns of `dbt.migration_objects` or by adding some runtime objects:

In [18]:
%%sql {PG}
select ddl1, ddl2 from dbt.migration_objects where type = 'TABLE' limit 1

1 rows affected.


ddl1,ddl2
"CREATE TABLE ""HR"".""EMPLOYEES"" ( ""EMPLOYEE_ID"" NUMBER(6,0), ""FIRST_NAME"" VARCHAR2(20), ""LAST_NAME"" VARCHAR2(25) CONSTRAINT ""EMP_LAST_NAME_NN"" NOT NULL ENABLE, ""EMAIL"" VARCHAR2(25) CONSTRAINT ""EMP_EMAIL_NN"" NOT NULL ENABLE, ""PHONE_NUMBER"" VARCHAR2(20), ""HIRE_DATE"" DATE CONSTRAINT ""EMP_HIRE_DATE_NN"" NOT NULL ENABLE, ""JOB_ID"" VARCHAR2(10) CONSTRAINT ""EMP_JOB_NN"" NOT NULL ENABLE, ""SALARY"" NUMBER(8,2), ""COMMISSION_PCT"" NUMBER(2,2), ""MANAGER_ID"" NUMBER(6,0), ""DEPARTMENT_ID"" NUMBER(4,0) )","CREATE UNIQUE INDEX ""HR"".""EMP_EMP_ID_PK"" ON ""HR"".""EMPLOYEES"" (""EMPLOYEE_ID"") ; CREATE UNIQUE INDEX ""HR"".""EMP_EMAIL_UK"" ON ""HR"".""EMPLOYEES"" (""EMAIL"") ;ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_EMP_ID_PK"" PRIMARY KEY (""EMPLOYEE_ID"") USING INDEX ""HR"".""EMP_EMP_ID_PK"" ENABLE;ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_EMAIL_UK"" UNIQUE (""EMAIL"") USING INDEX ""HR"".""EMP_EMAIL_UK"" ENABLE;ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_SALARY_MIN"" CHECK (salary > 0) ENABLE;ALTER TABLE ""HR"".""EMPLOYEES"" ADD SUPPLEMENTAL LOG DATA (ALL) COLUMNS;ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_DEPT_FK"" FOREIGN KEY (""DEPARTMENT_ID"") REFERENCES ""HR"".""DEPARTMENTS"" (""DEPARTMENT_ID"") ENABLE;ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_JOB_FK"" FOREIGN KEY (""JOB_ID"") REFERENCES ""HR"".""JOBS"" (""JOB_ID"") ENABLE;ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_MANAGER_FK"" FOREIGN KEY (""MANAGER_ID"") REFERENCES ""HR"".""EMPLOYEES"" (""EMPLOYEE_ID"") ENABLE;"


When errors are fixed we don't need to run the whole migration from the beginning. It's enough to reset only the problematic objects' `stage` field and restart the stage. Only those objects migration will be re-run.

The schema migration doesn't migrate constraints, indexes and triggers. They are migrated in `constraints` stage. This is done to make data migration run faster. The Liberatii Data Platform can also execute **multiple parallel instances** of each operation to signficantly improve transfer speeds. It is even possible to perform **parallel migrations for single tables** by splitting them.

The schema migration is started with same parameters as with `init` stage, but with `schema` as the stage name.

In [19]:
!curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "schema" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Again, waiting until this task is finished:


In [20]:
!curl -s -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"schema","status":"Running","messages":[{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"EMPLOYEES\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"LOCATIONS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"REGIONS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating PROCEDURE \"HR\".\"SECURE_DML\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating SEQUENCE \"HR\".\"DEPARTMENTS_SEQ\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating SEQUENCE \"HR\".\"LOCATIONS_SEQ\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"DEPARTMENTS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"COUNTRIES\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"JOB_HISTORY\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating PROCEDURE \"HR\".\"ADD_JOB_HISTORY\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating SEQ

In the `init` stage the framework also queries the sizes of each table. Using them we can get the most optimal strategy for copying the data.

## Data

So let's see the sizes:


In [21]:
%%sql {PG}
select name, pg_size_pretty(data_size), stage, error from dbt.migration_objects where type = 'TABLE' order by data_size desc

7 rows affected.


name,pg_size_pretty,stage,error
JOBS,64 kB,D,None
REGIONS,64 kB,D,None
DEPARTMENTS,64 kB,D,None
LOCATIONS,64 kB,D,None
EMPLOYEES,64 kB,D,None
JOB_HISTORY,64 kB,D,None
COUNTRIES,0 bytes,D,None



By analysing the sizes for different tables we can apply different strategies for different tables. But size the sizes are quire small for this schema we just use the default strategy everywhere.

Now starting the data migration operation, as all the operations before, but with `data` as `stage` field value:


In [22]:
!curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "data" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Awaiting the operation to be completed:

In [23]:
!curl -s -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"data","status":"Running","messages":[{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"JOBS\"in 0.177s(exec=0.1578s,copy-stmt=0.0009s,rows=0.0001s,write=0.0009s,set-state=0.0138s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"LOCATIONS\"in 0.061s(exec=0.0401s,copy-stmt=0.0009s,rows=0s,write=0.0008s,set-state=0.0131s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"REGIONS\"in 0.054s(exec=0.0391s,copy-stmt=0.0002s,rows=0s,write=0.0001s,set-state=0.0123s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"DEPARTMENTS\"in 0.054s(exec=0.0372s,copy-stmt=0.0004s,rows=0s,write=0.0004s,set-state=0.0126s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"EMPLOYEES\"in 0.07s(exec=0.0416s,copy-stmt=0.0042s,rows=0.0002s,write=0.0042s,set-state=0.0138s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"JOB_HISTORY\"in 0.055s(exec=0.038s,copy-stmt=0.0003s,rows=0s,w

## Constraints
This stage adds constraints, indexes and triggers to the data we've moved in the previous step:


In [24]:
!curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "constraints" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Awaiting the operation to be completed:

In [25]:
!curl -s -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"constraints","status":"Running","messages":[{"level":"Info","message":"^+👍^: ^gDONE!^ translating constraints for \"HR\".\"REGIONS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^ translating constraints for \"HR\".\"COUNTRIES\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^ translating constraints for \"HR\".\"LOCATIONS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^ translating constraints for \"HR\".\"JOBS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"LOC_COUNTRY_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"EMP_DEPARTMENT_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"EMP_JOB_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"EMP_NAME_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"JHIST_JOB_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"LOC_STATE_PROVINCE_IX\"\n"},{"level":"Info","message":

## Verification

With the data in place we can verify the migration across both databases to ensure they are the same. This is done by comparing hash sums run across data in each column:

In [26]:
!curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "check" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

In [27]:
!curl -s -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"check","status":"Running","messages":[{"level":"Info","message":"🤞 checking \"HR\".\"LOCATIONS\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"LOCATIONS\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"DEPARTMENTS\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"DEPARTMENTS\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"REGIONS\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"REGIONS\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"JOB_HISTORY\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"JOB_HISTORY\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"JOBS\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"JOBS\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"EMPLOYEES\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"EMPLOYEES\"\n"},{"level":"Info","messa

In [28]:
%%sql {PG}

select stage, type from dbt.migration_objects group by stage, type

5 rows affected.


stage,type
D,SEQUENCE
D,INDEX
D,VIEW
K,TABLE
D,PROCEDURE


This is it, now the whole database is migrated. Please see the guide document for the information about different migration options.

## Running queries

Let's now run a few queries in the both databases to see the results are identical. There is a `checksql` stage to do this automatically, but for the demo purposes we do this manually.

So first we check the databases are indeed Oracle and PostgreSQL:

In [29]:
%%sql {ORACLE}
select banner from v$version

0 rows affected.


banner
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production


In [30]:
%%sql {LGW}
select banner from v$version

1 rows affected.


BANNER
"PostgreSQL 14.9 (Debian 14.9-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit"


The output value on PostgreSQL can be configured to output exactly the same like Oracle if the application uses the data for some business logic selection.

Now let's run the same query in both databases. This query contains a number of Oracle-specific features:

* Concatenation using the double-pipe (||) operator
* The `NVL` and `DECODE` functions
* The join-plus operator and implicit joins

In [31]:
QUERY="""
SELECT first_name  || ' ' ||  last_name full_name,
        salary +
        NVL  (commission_pct, 0) sal_com,
        DECODE (NVL(e.department_id, -3),
                    60, d.department_name,
                    50, d.department_name,
                    30, d.department_name,
                    -3, 'UNKNOWN',
               'OTHER') dep,
        TO_CHAR(e.hire_date) hire_date
FROM employees e, departments d
WHERE d.department_id(+) = e.department_id
        AND e.last_name like 'G%'
ORDER BY first_name  ||  ' '  ||  last_name
"""

First running it on Oracle:

In [32]:
%%sql {ORACLE} 

{QUERY}

0 rows affected.


full_name,sal_com,dep,hire_date
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600,Shipping,13-JAN-08
Girard Geoni,2800,Shipping,03-FEB-08
Ki Gee,2400,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008,OTHER,17-AUG-02
Timothy Gates,2900,Shipping,11-JUL-06
William Gietz,8300,OTHER,07-JUN-02


And now running absolutely the same query but on PostgreSQL via Liberatii Gateway:

In [33]:
%%sql {LGW}

{QUERY}

8 rows affected.


FULL_NAME,SAL_COM,DEP,HIRE_DATE
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600.00,Shipping,13-JAN-08
Girard Geoni,2800.00,Shipping,03-FEB-08
Ki Gee,2400.00,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008.00,OTHER,17-AUG-02
Timothy Gates,2900.00,Shipping,11-JUL-06
William Gietz,8300.00,OTHER,07-JUN-02


The results are identical. 

The next steps are:

* Synchronise the databases using CDC so we avoid downtimes on switchover
* Replay workloads on a sandbox Oracle database and PostgreSQL via Liberatii Gateway, for testing correctness and performance


# Stage 3: Synchronisation

The next stage is to synchronise the Oracle and Postgres databases. This is done using a single API call to the Liberatii Data Platform but requires a user with greater privileges than that required for migration.

## Connection setup

The following API call adds new DBA user credentials to the API to allow synchronisation to take place:

In [34]:
DBA_USER='c##xstrm'
DBA_PASSWORD='xs'
!curl {API_PREFIX}/connection -H 'Content-Type: application/json' \
   -d '{{  "type":"Oracle",\
           "connectionString":"{ORACLE_CONN_STR}",\
           "user": "{DBA_USER}",\
           "password": "{DBA_PASSWORD}",\
           "id":1}}'

{"type":"Oracle","connectionString":"(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=oracle)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=pdborcl)))","user":"c##xstrm","password":"xs","id":1}

## Synchronisation setup

Once the connection is in place the synchronisation can be started in the same manner as previous stages.

In [35]:
!curl {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "postgres": 2, "lgw": 3, "stage": "sync" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

In [36]:
!curl -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"init","status":"Running","messages":[{"level":"Info","message":"\n^g^+DONE! 👍^:\n"},{"level":"Info","message":[["Type","D","K","Total"],["SEQUENCE","^g3^","","3"],["INDEX","^g11^","","11"],["VIEW","^g1^","","1"],["TABLE","","^g7^","7"],["PROCEDURE","^g2^","","2"],["^+Total:^:","^g^+17^","^g^+7^","^+24^"]]}],"progress":0}

## Synchronisation testing

The tables are now syncing, so we can test the result.

The following query selects the 3 most recent hires:

In [37]:
%%sql {LGW}

select employee_id, first_name  || ' ' ||  last_name full_name, hire_date
    from employees
    order by hire_date desc
    fetch first 3 rows only    

3 rows affected.


EMPLOYEE_ID,FULL_NAME,HIRE_DATE
167,Amit Banda,2008-04-21 00:00:00
173,Sundita Kumar,2008-04-21 00:00:00
166,Sundar Ande,2008-03-24 00:00:00


We can insert a new hire with a hire date after the most recent:

In [38]:
%%sql {ORACLE}

insert into HR.EMPLOYEES
    (EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID)
    values
    (300, 'Lewis', 'Carroll', 'LEWIS', '011.44.1346.123456', to_date('2008-04-22', 'YYYY-MM-DD'), 'SA_REP', 6300.00, 0.10, 147, 80)

1 rows affected.


[]

and make sure that the result is committed:

In [ ]:
%%capture
from IPython import get_ipython
import time

while True:
    result = get_ipython().run_cell_magic(
        'sql', LGW,
        "SELECT COUNT(*) FROM employees WHERE employee_id = 300")
    if result[0][0] != 0:
        break
    time.sleep(10)

Once this is complete the databases will now synchronise. The following query should show our new employee with the more recent hire date:

In [39]:
%%sql {LGW}

select employee_id, first_name  || ' ' ||  last_name full_name, hire_date
    from employees
    order by hire_date desc
    fetch first 3 rows only

3 rows affected.


EMPLOYEE_ID,FULL_NAME,HIRE_DATE
167,Amit Banda,2008-04-21 00:00:00
173,Sundita Kumar,2008-04-21 00:00:00
166,Sundar Ande,2008-03-24 00:00:00


# Stage 4. Replay testing

Replay testing is performed using the Oracle Workload Replay tool. In a production setup this is performed by:
1. Capturing workload on a production Oracle database
   - It is possible to filter to the resulting capture to remove senstive data
2. Transferring this capture to a testing database
3. Running the Oracle workload-replay tool to run the queries and obtain statistics

In this demonstration the same Oracle database will be used as the source of the capture and its target.

**Note:** The Notebook %%sql magic does not support transaction blocks so the `sqlplus` tool is referenced directly.

## 1. Capture

The first stage is to capture a workload.

In [41]:
%%script sqlplus "{USER}/{PSWD}@{ORACLE_HOST}:{ORACLE_PORT}/{DB}"
CREATE OR REPLACE DIRECTORY REPLAY_DIR AS '/tmp/replay_dir';
BEGIN
DBMS_WORKLOAD_CAPTURE.START_CAPTURE (
    name             => 'CAPTURE',
    dir              => 'REPLAY_DIR',
    duration         => NULL,
    capture_sts      => TRUE,
    sts_cap_interval => 300);
END;
/
exit


SQL*Plus: Release 21.0.0.0.0 - Production on Wed Aug 30 15:28:43 2023
Version 21.10.0.0.0

Copyright (c) 1982, 2022, Oracle.  All rights reserved.

Last Successful login time: Wed Aug 30 2023 15:28:20 +00:00

Connected to:
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.3.0.0.0

SQL> 
Directory created.

SQL>   2    3    4    5    6    7    8    9  
PL/SQL procedure successfully completed.

SQL> Disconnected from Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.3.0.0.0


Once the capture has started we can begin running queries:

In [42]:
%%sql {ORACLE}

{QUERY}

0 rows affected.


full_name,sal_com,dep,hire_date
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600,Shipping,13-JAN-08
Girard Geoni,2800,Shipping,03-FEB-08
Ki Gee,2400,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008,OTHER,17-AUG-02
Timothy Gates,2900,Shipping,11-JUL-06
William Gietz,8300,OTHER,07-JUN-02


Finally, we can finish the workload capture:

In [43]:
%%script sqlplus "{USER}/{PSWD}@{ORACLE_HOST}:{ORACLE_PORT}/{DB}"
BEGIN
    DBMS_WORKLOAD_CAPTURE.FINISH_CAPTURE ();
END;
/
exit


SQL*Plus: Release 21.0.0.0.0 - Production on Wed Aug 30 15:28:57 2023
Version 21.10.0.0.0

Copyright (c) 1982, 2022, Oracle.  All rights reserved.

Last Successful login time: Wed Aug 30 2023 15:28:51 +00:00

Connected to:
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.3.0.0.0

SQL>   2    3    4  
PL/SQL procedure successfully completed.

SQL> Disconnected from Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.3.0.0.0


## 2. Transfer, preparation

Once the capture has been created we can initialise the capture for use by the Workload Replay Client. In a production environment this step would be performed on the testing database after the workload was transferred across. 

In [44]:
%%script sqlplus "{USER}/{PSWD}@{ORACLE_HOST}:{ORACLE_PORT}/{DB}"
BEGIN
    DBMS_WORKLOAD_REPLAY.PROCESS_CAPTURE (
        capture_dir => 'REPLAY_DIR');
    DBMS_WORKLOAD_REPLAY.INITIALIZE_REPLAY(
        replay_name=> 'CAPTURE',
        replay_dir => 'REPLAY_DIR');
    DBMS_WORKLOAD_REPLAY.PREPARE_REPLAY (
        synchronization => TRUE,
        capture_sts => TRUE,
        sts_cap_interval => 300);
END;
/
exit


SQL*Plus: Release 21.0.0.0.0 - Production on Wed Aug 30 15:29:25 2023
Version 21.10.0.0.0

Copyright (c) 1982, 2022, Oracle.  All rights reserved.

Last Successful login time: Wed Aug 30 2023 15:28:58 +00:00

Connected to:
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.3.0.0.0

SQL>   2    3    4    5    6    7    8    9   10   11   12  
PL/SQL procedure successfully completed.

SQL> Disconnected from Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.3.0.0.0


## 3. Workload testing

Once the workload has been transferred and prepared it can be tested using the client tool. Here a new user "tester001" is used with permissions to run a workload on the database.

The following operation will initialise the `wrc` tool and wait for the database to be signalled to start the replay.

In [45]:
TEST_USER='tester001'
TEST_PASSWORD='tester001'

!curl -s -XPOST {API_PREFIX}/config -H 'Content-Type: application/json' \
  -d '{{"replayDir":"/tmp/replay_dir","verbose":4}}'

!curl -s -XPOST {API_PREFIX}/connection -H 'Content-Type: application/json' \
  -d '{{  "type":"Oracle",\
           "connectionString":"{ORACLE_CONN_STR}",\
           "user": "{TEST_USER}",\
           "password": "{TEST_PASSWORD}",\
           "id":1}}'

!curl -s -XPOST {API_PREFIX}/operation -H 'Content-Type: application/json' \
  -d '{{"oracle": 1, "postgres": 2, "lgw": 3, "stage": "replay"}}'

{"message":"Config has been set successfully","config":{"dataOnePass":true,"verbose":4,"useCopy":true,"dataIterations":-1,"useWrapper":true,"useNative":false,"useUnlogged":false,"stat":true,"statDB":false,"rowsBuf":1000,"lightCheck":false,"dataChunkSize":-1,"bigTablesFirst":true,"debReverseOrder":false,"cli":true,"rmStagingFiles":true,"parTables":true,"hashType":"murmur","simulateUnsupportedTypes":true,"blobStreams":true,"clobStreams":false,"blobImmed":false,"clobImmed":false,"dumpCopy":false,"dryRun":false,"noBlobs":false,"idCol":"rowid","idColByTable":{},"activeSqlFiles":[],"numTries":10,"maxWait":60000,"ignoreTrim":true,"removeLastFetchFirst":false,"checkMetaData":true,"users":[],"linkedServers":[],"stages":{},"workloadsFiles":"","eraseOnInit":false,"debezium":"http://kafka-connect:8083/","replayDir":"/tmp/replay_dir"}}{"type":"Oracle","connectionString":"(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=oracle)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=pdborcl)))","user":"tester001","password

The replay can be started as follows:

In [46]:
%%script sqlplus "{USER}/{PSWD}@{ORACLE_HOST}:{ORACLE_PORT}/{DB}"
BEGIN
  FOR i IN 1..60 LOOP
    BEGIN
      DBMS_WORKLOAD_REPLAY.START_REPLAY();
      EXIT;
    EXCEPTION WHEN OTHERS THEN
      IF INSTR(SQLERRM, 'ORA-20223: No replay clients have connected yet') = 0 THEN
        RAISE;
      END IF;
      DBMS_LOCK.SLEEP(1);
    END;
  END LOOP;
END;
/
exit


SQL*Plus: Release 21.0.0.0.0 - Production on Wed Aug 30 15:30:15 2023
Version 21.10.0.0.0

Copyright (c) 1982, 2022, Oracle.  All rights reserved.

Last Successful login time: Wed Aug 30 2023 15:29:25 +00:00

Connected to:
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.3.0.0.0

SQL>   2    3    4    5    6    7    8    9   10   11   12   13   14   15  BEGIN
*
ERROR at line 1:
ORA-20223: Database not in PREPARE state; issue PREPARE_REPLAY() before
START_REPLAY()
ORA-06512: at "SYS.DBMS_WORKLOAD_REPLAY_I", line 6967
ORA-06512: at "SYS.DBMS_WORKLOAD_REPLAY", line 169
ORA-06512: at line 13


SQL> Disconnected from Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.3.0.0.0


# 4. Results

We now wait for the replay operation to complete:

In [47]:
!curl -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

^C


This should produce a table of results that compare queries from the workload capture running in Oracle to those running via Liberatii Gateway.

In [49]:
%%sql {PG}
select SUBSTRING(statement, 1, 40), oracle_duration, liberatii_duration from dbt.replay

7 rows affected.


substring,oracle_duration,liberatii_duration
"select v.conn_id, v.capture_conn, v.re",6,54
"select v.conn_id, v.capture_conn, v.re",75,18
"select sys_context( 'userenv', 'current_",5,18
SELECT value FROM v$parameter WHERE name,9,16
SELECT 1.1 FROM DUAL,1,14
SELECT first_name || ' ' || last_name,24,25
"SELECT DECODE(USER, 'XS$NULL', XS_SYS_C",5,19


# In conclusion

We have used the Liberatii Data Platform to migrate the HR schema and its associated data from an Oracle database to a PostgreSQL database virtualized using the Liberatii Gateway.

Most importantly:

* We made no changes to the code
* We kept the resulting database up to date using a Change-Data-Capture pipeline
* We tested a saved workload against an up-to-date virtualized copy, obviating the need for additional tests

This means no time or money needs to be spent finding and rewriting code or writing new test cases.